
# Objetivo 2: Generación de modelos

Pipeline completo para la predicción del crecimiento de datos de biodiversidad en GBIF.

Este script implementa el flujo de trabajo de principio a fin para modelar datos de panel
de series temporales, incluyendo:
1.  Carga datos de PA_dataAnalysis y preparación
2.  Ingeniería de características temporales (lags y ventanas móviles).
3.  Un marco de validación cruzada robusto para series de tiempo (ventana expansiva).
4.  Preprocesamiento (imputación y escalado) dentro del bucle de validación para evitar fuga de datos.
5.  Entrenamiento y evaluación comparativa de cuatro modelos:
    - Prophet.
    - Random Forest.
    - XGBoost.
    - Red Neuronal LSTM (para modelado secuencial).
6.  Selección del mejor modelo basado en métricas de rendimiento (MAE, RMSE, R²).
7.  Reentrenamiento del modelo final y generación de pronósticos para Colombia hasta 2030
    bajo dos escenarios de políticas.

In [8]:
# =============================================================================
# 1. IMPORTACIÓN DE LIBRERÍAS Y CONFIGURACIÓN INICIAL
# =============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from tqdm import tqdm

# Preprocesamiento y modelado de Scikit-Learn
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Modelos especializados
import xgboost as xgb
from statsmodels.tsa.statespace.sarimax import SARIMAX



# Configuraciones generales
warnings.filterwarnings('ignore')
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (15, 7)

/Users/ricardoortiz/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/ricardoortiz/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [ ]:
# Modelado de Deep Learning con TensorFlow/Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [16]:
#Carga de datos

url = "https://raw.githubusercontent.com/rortizgeo/Maestria_CD_Proyecto-Aplicado/main/Data_final.csv"
Data_final = pd.read_csv(url)

In [25]:
# =============================================================================
# 2. INGENIERÍA DE CARACTERÍSTICAS TEMPORALES
# =============================================================================
print("\nPaso 2: Realizando ingeniería de características temporales (versión combinada)...")

"""
    Crea características de retardo (lag) y de ventana móvil para el conjunto de datos preparado previamente.
    Las operaciones se agrupan por país para evitar la fuga de datos entre series.
"""

TARGET = 'occurrenceCount_publisher'

def create_temporal_features_safe(data, features_to_lag, lags=[1, 2, 3,4,5,6,7,8,9,10], roll_windows=[3, 5, 7, 10]):
    """
    Genera características temporales (lags y rolling) por país,
    evitando fuga de datos con shift(1).
    
    Parámetros
    ----------
    data : pd.DataFrame
        Dataset con al menos 'country' y variables numéricas.
    features_to_lag : list
        Lista de columnas numéricas a transformar.
    lags : list
        Lista de retardos (ej. [1,2]).
    roll_windows : list
        Lista de ventanas móviles (ej. [3,5]).
        
    Retorna
    -------
    DataFrame con nuevas columnas de lags y rolling.
    """
    
    df_copy = data.copy()
    
    for feature in features_to_lag:
        for lag in lags:
            df_copy[f'{feature}_lag{lag}'] = (
                df_copy.groupby('country')[feature].shift(lag)
            )
        for w in roll_windows:
            df_copy[f'{feature}_rollmean{w}'] = (
                df_copy.groupby('country')[feature].shift(1).rolling(w).mean()
            )
            df_copy[f'{feature}_rollstd{w}'] = (
                df_copy.groupby('country')[feature].shift(1).rolling(w).std()
            )
    
    return df_copy


# ===========================
# Uso del código
# ===========================
# Ejemplo: variables socioeconómicas y target
features_to_lag = [
    "occurrenceCount_publisher", "pib_per_capita",
    "gasto_educacion_gobierno", "gasto_educacion_pib"
]

# Crear dataset con nuevas features
df_featured = create_temporal_features_safe(
    Data_final,
    features_to_lag=features_to_lag,
    lags=[1, 2,3,4,5,6,7,8,9,10],
    roll_windows=[3, 5, 7, 10]
)

print("Ingeniería de características completada.")
print(df_featured.head(10))



Paso 2: Realizando ingeniería de características temporales (versión combinada)...
Ingeniería de características completada.
   country  year  superficie_total_km2  areas_protegidas  pib_per_capita  \
0  Andorra  2007                 470.0               NaN    48336.228333   
1  Andorra  2008                 470.0               NaN    49132.515708   
2  Andorra  2009                 470.0               NaN    43975.018367   
3  Andorra  2010                 470.0               NaN    42746.830953   
4  Andorra  2011                 470.0               NaN    46657.156117   
5  Andorra  2012                 470.0               NaN    41500.543579   
6  Andorra  2013                 470.0               9.2    42470.316116   
7  Andorra  2014                 470.0              18.2    44369.659691   
8  Andorra  2015                 470.0              21.1    38654.934720   
9  Andorra  2016                 470.0              21.1    40129.819201   

   gasto_educacion_gobierno  gasto_ed

In [ ]:
# =============================================================================
# 3. PREPARACIÓN PARA EL MODELADO Y VALIDACIÓN
# =============================================================================
print("\nPaso 3: Preparando el marco de validación y los datos para el modelado...")

# Definir variable objetivo
TARGET = 'occurrenceCount_publisher'

# Definir variables predictoras (ajusta según tus columnas disponibles)
features = [
    'PC1', 'PC2', 'pib_per_capita', 'gasto_educacion_gobierno',
    'gasto_educacion_pib', 'superficie_total_km2', 'areas_protegidas',
    f"{TARGET}_lag1", f"{TARGET}_lag2", f"{TARGET}_rollmean3", f"{TARGET}_rollstd3"
]

# Filtrar las variables que realmente existen en el DataFrame
features = [f for f in features if f in df_featured.columns]

# Definir X (predictoras) e y (objetivo)
X = df_featured[features].copy()
y = df_featured[TARGET].copy()

# Configurar validación cruzada para series de tiempo
n_splits = 5  # número de pliegues
tscv = TimeSeriesSplit(n_splits=n_splits)

# Diccionario para almacenar resultados
results = {
    'Prophet': None,
    'RandomForest': None,
    'XGBoost': None,
    'LSTM': None
}

print("Features seleccionadas:", features)
print("X shape:", X.shape)
print("y shape:", y.shape)


In [ ]:
# =============================================================================
# 5. BUCLE DE ENTRENAMIENTO Y EVALUACIÓN DE MODELOS
# =============================================================================
print("\nPaso 4: Iniciando el bucle de entrenamiento y validación de modelos...")

for fold, (train_year_idx, test_year_idx) in enumerate(tscv.split(unique_years)):
    print(f"\n===== FOLD {fold + 1}/{n_splits} =====")
    
    # Identificar los años de entrenamiento y prueba
    train_years = unique_years[train_year_idx]
    test_years = unique_years[test_year_idx]
    print(f"Años de entrenamiento: {train_years.min()} - {train_years.max()}")
    print(f"Años de prueba: {test_years.min()} - {test_years.max()}")

    # Dividir los datos en entrenamiento y prueba según los años
    train_indices = X[X['year'].isin(train_years)].index
    test_indices = X[X['year'].isin(test_years)].index
    
    X_train, X_test = X.loc[train_indices], X.loc[test_indices]
    y_train, y_test = y.loc[train_indices], y.loc[test_indices]

    # --- Preprocesamiento DENTRO del bucle para evitar fuga de datos ---
    # Guardar los nombres de las columnas y los países para después
    X_train_countries = X_train['country']
    X_test_countries = X_test['country']
    X_train_columns = X_train.drop(columns=['country']).columns
    
    # Separar 'country' antes de imputar y escalar
    X_train_numeric = X_train.drop(columns=['country'])
    X_test_numeric = X_test.drop(columns=['country'])

    # Imputación de valores faltantes
    imputer = IterativeImputer(max_iter=10, random_state=42)
    X_train_imputed = imputer.fit_transform(X_train_numeric)
    X_test_imputed = imputer.transform(X_test_numeric)

    # Escalado de características
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_imputed)
    X_test_scaled = scaler.transform(X_test_imputed)
    
    # Convertir de nuevo a DataFrame para facilidad de uso
    X_train_processed = pd.DataFrame(X_train_scaled, columns=X_train_columns, index=X_train.index)
    X_test_processed = pd.DataFrame(X_test_scaled, columns=X_train_columns, index=X_test.index)
    
    # Añadir de nuevo la columna 'country'
    X_train_processed['country'] = X_train_countries
    X_test_processed['country'] = X_test_countries

    # --- Modelo 1: SARIMAX (Línea Base Estadística) ---
    print("Entrenando SARIMAX...")
    sarimax_preds =
    y_test_sarimax =
    
    # Se ajusta un modelo por cada país en el conjunto de prueba
    for country in tqdm(X_test_processed['country'].unique(), desc="SARIMAX per country"):
        train_country_data = df_featured[df_featured['country'] == country][df_featured['year'].isin(train_years)]
        test_country_data = df_featured[df_featured['country'] == country][df_featured['year'].isin(test_years)]
        
        if not train_country_data.empty and not test_country_data.empty:
            endog = train_country_data
            exog_cols = [c for c in features if c not in ['year', 'country']]
            exog_train = train_country_data[exog_cols]
            exog_test = test_country_data[exog_cols]
            
            try:
                # Usamos un orden simple (p,d,q)=(1,1,1) para la demostración
                model = SARIMAX(endog, exog=exog_train, order=(1, 1, 1), seasonal_order=(0, 0, 0, 0))
                results_sarimax = model.fit(disp=False)
                forecast = results_sarimax.get_forecast(steps=len(test_country_data), exog=exog_test)
                sarimax_preds.extend(forecast.predicted_mean.values)
                y_test_sarimax.extend(test_country_data.values)
            except Exception as e:
                # Si un modelo falla, predecimos la última observación conocida
                sarimax_preds.extend([endog.iloc[-1]] * len(test_country_data))
                y_test_sarimax.extend(test_country_data.values)

    if y_test_sarimax:
        mae = mean_absolute_error(y_test_sarimax, sarimax_preds)
        rmse = np.sqrt(mean_squared_error(y_test_sarimax, sarimax_preds))
        r2 = r2_score(y_test_sarimax, sarimax_preds)
        results.append({'MAE': mae, 'RMSE': rmse, 'R2': r2})
        print(f"SARIMAX - MAE: {mae:.2f}, RMSE: {rmse:.2f}, R2: {r2:.2f}")

    # --- Modelos de Machine Learning (usando datos procesados sin 'country') ---
    X_train_ml = X_train_processed.drop(columns=['country'])
    X_test_ml = X_test_processed.drop(columns=['country'])

    # --- Modelo 2: Random Forest ---
    print("Entrenando Random Forest...")
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    rf_model.fit(X_train_ml, y_train)
    rf_preds = rf_model.predict(X_test_ml)
    mae = mean_absolute_error(y_test, rf_preds)
    rmse = np.sqrt(mean_squared_error(y_test, rf_preds))
    r2 = r2_score(y_test, rf_preds)
    results.append({'MAE': mae, 'RMSE': rmse, 'R2': r2})
    print(f"Random Forest - MAE: {mae:.2f}, RMSE: {rmse:.2f}, R2: {r2:.2f}")

    # --- Modelo 3: XGBoost ---
    print("Entrenando XGBoost...")
    xgb_model = xgb.XGBRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    xgb_model.fit(X_train_ml, y_train)
    xgb_preds = xgb_model.predict(X_test_ml)
    mae = mean_absolute_error(y_test, xgb_preds)
    rmse = np.sqrt(mean_squared_error(y_test, xgb_preds))
    r2 = r2_score(y_test, xgb_preds)
    results.append({'MAE': mae, 'RMSE': rmse, 'R2': r2})
    print(f"XGBoost - MAE: {mae:.2f}, RMSE: {rmse:.2f}, R2: {r2:.2f}")

    # --- Modelo 4: LSTM ---
    print("Entrenando LSTM...")
    
    def create_lstm_dataset(X, y, countries, look_back=3):
        dataX, dataY =,
        for country in countries.unique():
            X_country = X[countries == country]
            y_country = y[countries == country]
            if len(X_country) > look_back:
                for i in range(len(X_country) - look_back):
                    a = X_country.iloc[i:(i + look_back)].values
                    dataX.append(a)
                    dataY.append(y_country.iloc[i + look_back])
        return np.array(dataX), np.array(dataY)

    look_back = 3
    X_train_lstm, y_train_lstm = create_lstm_dataset(X_train_ml, y_train, X_train_processed['country'], look_back)
    X_test_lstm, y_test_lstm = create_lstm_dataset(X_test_ml, y_test, X_test_processed['country'], look_back)

    if X_train_lstm.shape > 0 and X_test_lstm.shape > 0:
        # Definir la arquitectura del modelo LSTM
        lstm_model = Sequential(, X_train_lstm.shape[2])),
            Dropout(0.2),
            Dense(1)
        ])
        lstm_model.compile(optimizer='adam', loss='mean_squared_error')
        
        # Entrenar el modelo
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        lstm_model.fit(X_train_lstm, y_train_lstm, epochs=50, batch_size=32, 
                       validation_data=(X_test_lstm, y_test_lstm), 
                       callbacks=[early_stopping], verbose=0)
        
        # Realizar predicciones
        lstm_preds = lstm_model.predict(X_test_lstm).flatten()
        
        mae = mean_absolute_error(y_test_lstm, lstm_preds)
        rmse = np.sqrt(mean_squared_error(y_test_lstm, lstm_preds))
        r2 = r2_score(y_test_lstm, lstm_preds)
        results.append({'MAE': mae, 'RMSE': rmse, 'R2': r2})
        print(f"LSTM - MAE: {mae:.2f}, RMSE: {rmse:.2f}, R2: {r2:.2f}")

In [ ]:
# =============================================================================
# 6. ANÁLISIS COMPARATIVO Y SELECCIÓN DEL MODELO
# =============================================================================
print("\nPaso 5: Analizando y comparando los resultados de los modelos...")

# Convertir los resultados a un DataFrame para fácil visualización
summary_list =
for model_name, metrics_list in results.items():
    if metrics_list:
        df_metrics = pd.DataFrame(metrics_list)
        summary_list.append({
            'Model': model_name,
            'MAE_mean': df_metrics['MAE'].mean(),
            'MAE_std': df_metrics['MAE'].std(),
            'RMSE_mean': df_metrics.mean(),
            'RMSE_std': df_metrics.std(),
            'R2_mean': df_metrics.mean(),
            'R2_std': df_metrics.std()
        })

summary_df = pd.DataFrame(summary_list).set_index('Model')
print("\nResumen del rendimiento de los modelos (promedio de los pliegues de CV):")
print(summary_df)

# Seleccionar el mejor modelo (basado en el RMSE más bajo)
best_model_name = summary_df.idxmin()
print(f"\nMejor modelo seleccionado: {best_model_name}")

In [ ]:
# =============================================================================
# 7. REENTRENAMIENTO DEL MODELO FINAL Y PRONÓSTICO PARA COLOMBIA
# =============================================================================
print(f"\nPaso 6: Reentrenando el modelo {best_model_name} con todos los datos...")

# Preparar todos los datos para el reentrenamiento
X_full_numeric = X.drop(columns=['country'])
imputer_final = IterativeImputer(max_iter=10, random_state=42)
X_full_imputed = imputer_final.fit_transform(X_full_numeric)
scaler_final = StandardScaler()
X_full_scaled = scaler_final.fit_transform(X_full_imputed)
X_full_processed = pd.DataFrame(X_full_scaled, columns=X_full_numeric.columns, index=X.index)

# Reentrenar el modelo XGBoost
final_model = xgb.XGBRegressor(n_estimators=100, random_state=42, n_jobs=-1)
final_model.fit(X_full_processed, y)

print("Modelo final entrenado. Generando pronósticos para Colombia...")

# --- Creación de escenarios futuros para Colombia ---
colombia_last_data = df_featured[df_featured['country'] == 'Colombia'].iloc[-1]
future_years = range(2023, 2031)
future_df = pd.DataFrame()

for year in future_years:
    future_row = colombia_last_data.copy()
    future_row['year'] = year
    future_df = pd.concat(, ignore_index=True)

# Escenario A: "Business as Usual" (extrapolación simple)
for col in:
    # Simple extrapolación lineal para la tendencia
    last_val = colombia_last_data[col]
    trend = (colombia_last_data[col] - df_featured[df_featured['country'] == 'Colombia'][col].iloc[-5:].mean()) / 5
    future_df[col] = [last_val + trend * i for i in range(1, len(future_years) + 1)]

# Escenario B: "Inversión Estratégica" (aumento del 10% en gasto en educación)
future_df_optimistic = future_df.copy()
future_df_optimistic['gasto_educacion_pib'] *= 1.10

# Función para predecir recursivamente
def generate_forecast(model, initial_data, future_template, scaler, imputer):
    history = initial_data.copy()
    predictions =
    
    for i in range(len(future_template)):
        # Preparar la fila para la predicción actual
        current_step_features = future_template.iloc[[i]]
        
        # Actualizar lags y rolling stats con el último dato conocido (de history)
        last_known = history.iloc[-1]
        for feature in features_to_lag:
            current_step_features[f'{feature}_lag_1'] = last_known[feature]
            # Para rolling stats, usamos los últimos datos de history
            rolling_window = pd.concat()])[feature]
            current_step_features[f'{feature}_rolling_mean_3'] = rolling_window.mean()
            current_step_features[f'{feature}_rolling_std_3'] = rolling_window.std()

        # Preprocesar la fila
        current_step_numeric = current_step_features[X_full_numeric.columns]
        current_step_imputed = imputer.transform(current_step_numeric)
        current_step_scaled = scaler.transform(current_step_imputed)
        
        # Predecir
        prediction = model.predict(current_step_scaled)
        predictions.append(prediction)
        
        # Actualizar 'history' con la nueva predicción para el siguiente paso
        new_row = current_step_features.copy()
        new_row = prediction
        history = pd.concat([history, new_row], ignore_index=True)
        
    return predictions

# Generar pronósticos para ambos escenarios
forecast_base = generate_forecast(final_model, df_featured[df_featured['country'] == 'Colombia'], future_df, scaler_final, imputer_final)
forecast_optimistic = generate_forecast(final_model, df_featured[df_featured['country'] == 'Colombia'], future_df_optimistic, scaler_final, imputer_final)

# --- Visualización de los resultados ---
plt.figure(figsize=(18, 9))
# Datos históricos
plt.plot(df_featured[df_featured['country'] == 'Colombia']['year'], 
         df_featured[df_featured['country'] == 'Colombia'], 
         label='Histórico - SiB Colombia', color='black', marker='o')
# Pronóstico Base
plt.plot(future_years, forecast_base, label='Pronóstico Base ("Business as Usual")', color='blue', marker='x', linestyle='--')
# Pronóstico Optimista
plt.plot(future_years, forecast_optimistic, label='Pronóstico Optimista (↑ Gasto Educación)', color='green', marker='^', linestyle='--')

plt.title('Pronóstico de Crecimiento de Publicación de Datos para SiB Colombia (2023-2030)', fontsize=16)
plt.xlabel('Año', fontsize=12)
plt.ylabel('Número de Registros Publicados (occurrenceCount_publisher)', fontsize=12)
plt.legend()
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.show()

print("\nProceso completado.")